# Praca domowa 1


## Stworzenie modelu

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import FunctionTransformer
from sklearn.metrics import accuracy_score

import pickle
import dalex as dx

In [2]:
# Wczytanie i przygotowanie danych 
full_data = pd.read_csv("data/hotel_bookings.csv")
full_data["agent"] = full_data["agent"].astype(str)
treshold = 0.005 * len(full_data)
agents_to_change = full_data['agent'].value_counts()[full_data['agent'].value_counts() < treshold].index
full_data.loc[full_data["agent"].isin(agents_to_change), "agent"] = "other"

countries_to_change = full_data['country'].value_counts()[full_data['country'].value_counts() < treshold].index
full_data.loc[full_data["country"].isin(countries_to_change), "country"] = "other"


# Określenie cech uwzględnionych w modelu
num_features = ["lead_time", "arrival_date_week_number",
                "stays_in_weekend_nights", "stays_in_week_nights", 
                "adults", "previous_cancellations",
                "previous_bookings_not_canceled",
                "required_car_parking_spaces", "total_of_special_requests", 
                "adr", "booking_changes"]

cat_features = ["hotel", "market_segment", "country", 
                "reserved_room_type",
                "customer_type", "agent"]

features = num_features + cat_features

# Podział na zmienne wyjaśniające i target
X = full_data.drop(["is_canceled"], axis=1)[features]
y = full_data["is_canceled"]

# Preprocessing
num_transformer = SimpleImputer(strategy="constant")

cat_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="constant", fill_value="Unknown")),
    ("onehot", OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(transformers=[("num", num_transformer, num_features),
                                               ("cat", cat_transformer, cat_features)],
                                remainder = 'passthrough')

In [3]:
# Stworzenie modelu
rf_model_enh = RandomForestClassifier(n_estimators=160,
                               max_features=0.4,
                               min_samples_split=2,
                               n_jobs=-1,
                               random_state=42)

model_pipe = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', rf_model_enh)])

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2, random_state=42)

model_pipe.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num',
                                                  SimpleImputer(strategy='constant'),
                                                  ['lead_time',
                                                   'arrival_date_week_number',
                                                   'stays_in_weekend_nights',
                                                   'stays_in_week_nights',
                                                   'adults',
                                                   'previous_cancellations',
                                                   'previous_bookings_not_canceled',
                                                   'required_car_parking_spaces',
                                                   'total_of_special_requests',
                                                   'adr', 'booking_changes']),
    

## Wyjaśnianie

In [4]:
explainer = dx.Explainer(model_pipe, X_train, y_train)

Preparation of a new explainer is initiated

  -> data              : 95512 rows 17 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 95512 values
  -> model_class       : sklearn.ensemble._forest.RandomForestClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_proba_default at 0x00000252D7A555E0> will be used (default)
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.0, mean = 0.371, max = 1.0
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.943, mean = -0.00182, max = 0.957
  -> model_info        : package sklearn

A new explainer has been created!


In [5]:
predicts = explainer.predict(X_train)

### 1. Predykcja dla obserwacji nr 1777

In [21]:
predicts[1777]

0.9625

Model przewiduje anulację rezerwacji dla rekordu 1777

### 2. Dekompozycja predykcji modelu dla wybranych 6 obserwacji, w tym obserwacji 1777 (3 pierwsze z realną wartością 0, kolejne 3 z wartością 1)

In [9]:
pps_list = []
ids = [289, 999, 1836, 1248, 1777, 2626]
for id in ids:
    pps_list.append(explainer.predict_parts(X_train.iloc[id,:]))

In [13]:
pps_list[4] ## 1777

,variable_name,variable_value,variable,cumulative,contribution,sign,position,label
0,intercept,1,intercept,0.370915,0.370915,1.0,17,RandomForestClassifier
1,total_of_special_requests,2.0,total_of_special_requests = 2.0,0.225115,-0.145800,-1.0,16,RandomForestClassifier
2,country,PRT,country = PRT,0.312910,0.087795,1.0,15,RandomForestClassifier
3,agent:hotel,240.0:Resort Hotel,agent:hotel = 240.0:Resort Hotel,0.420854,0.107944,1.0,14,RandomForestClassifier
4,customer_type,Transient,customer_type = Transient,0.461545,0.040691,1.0,13,RandomForestClassifier
5,adr,48.0,adr = 48.0,0.484718,0.023173,1.0,12,RandomForestClassifier
6,stays_in_week_nights,5.0,stays_in_week_nights = 5.0,0.553154,0.068436,1.0,11,RandomForestClassifier
7,market_segment,Online TA,market_segment = Online TA,0.648691,0.095537,1.0,10,RandomForestClassifier
8,previous_cancellations,0.0,previous_cancellations = 0.0,0.642768,-0.005923,-1.0,9,RandomForestClassifier
9,required_car_parking_spaces,0.0,required_car_parking_spaces = 0.0,0.672644,0.029877,1.0,8,RandomForestClassifier


In [14]:
pps_list[4].plot()

Jak widzimy na powyższym wykresie największy wpływ na wyliczoną predykcję miał:  kraj, z którego pochodził rezerwujący, agent w połączeniu z hotelem, a także sposób rezerwacji i liczba nocy, na którą rezerwację złożył gość. Z kolei zmienną, która sugerowała, że gość nie anuluje rezerwacji była liczba specjalnych próśb.

In [15]:
pp_shap = explainer.predict_parts(X_train.iloc[1777,:], type='shap')

In [17]:
pp_shap.plot()

Możemy zauważyć, że wpływ zmiennych jest inny dla innych rodzajów wykresów. Wynika to prawdopodobnie z tego, że w metodzie break down jest ściśle zdefiniowana kolejność zmiennych, które mogą na siebie oddziaływać. 
Metoda shap zauważalnie dłużej się liczy.

### 3. Wybór obserwacji z innymi najważniejszymi zmiennymi

In [31]:
pps_list[0].plot()
pps_list[1].plot()

In [28]:
pps_list[0] ## 289

,variable_name,variable_value,variable,cumulative,contribution,sign,position,label
0,intercept,1,intercept,0.370915,0.370915,1.0,16,RandomForestClassifier
1,lead_time,0.0,lead_time = 0.0,0.140108,-0.230807,-1.0,15,RandomForestClassifier
2,customer_type:country,Contract:PRT,customer_type:country = Contract:PRT,0.226374,0.086266,1.0,14,RandomForestClassifier
3,total_of_special_requests,0.0,total_of_special_requests = 0.0,0.247657,0.021283,1.0,13,RandomForestClassifier
4,agent,9.0,agent = 9.0,0.234065,-0.013592,-1.0,12,RandomForestClassifier
5,arrival_date_week_number,35.0,arrival_date_week_number = 35.0,0.235593,0.001529,1.0,11,RandomForestClassifier
6,market_segment,Online TA,market_segment = Online TA,0.207733,-0.027860,-1.0,10,RandomForestClassifier
7,previous_cancellations,0.0,previous_cancellations = 0.0,0.182716,-0.025017,-1.0,9,RandomForestClassifier
8,required_car_parking_spaces,0.0,required_car_parking_spaces = 0.0,0.190733,0.008017,1.0,8,RandomForestClassifier
9,booking_changes,0.0,booking_changes = 0.0,0.206981,0.016248,1.0,7,RandomForestClassifier


In [29]:
pps_list[1] ## 999

,variable_name,variable_value,variable,cumulative,contribution,sign,position,label
0,intercept,1,intercept,0.370915,0.370915,1.0,17,RandomForestClassifier
1,required_car_parking_spaces,1.0,required_car_parking_spaces = 1.0,0.064389,-0.306526,-1.0,16,RandomForestClassifier
2,total_of_special_requests,2.0,total_of_special_requests = 2.0,0.038572,-0.025816,-1.0,15,RandomForestClassifier
3,lead_time,9.0,lead_time = 9.0,0.045346,0.006774,1.0,14,RandomForestClassifier
4,country,PRT,country = PRT,0.049205,0.003859,1.0,13,RandomForestClassifier
5,agent:hotel,240.0:Resort Hotel,agent:hotel = 240.0:Resort Hotel,0.037280,-0.011925,-1.0,12,RandomForestClassifier
6,customer_type,Transient,customer_type = Transient,0.029913,-0.007367,-1.0,11,RandomForestClassifier
7,market_segment,Online TA,market_segment = Online TA,0.023172,-0.006741,-1.0,10,RandomForestClassifier
8,previous_cancellations,0.0,previous_cancellations = 0.0,0.005063,-0.018110,-1.0,9,RandomForestClassifier
9,adr,94.5,adr = 94.5,0.003691,-0.001372,-1.0,8,RandomForestClassifier


Zauważmy że w przypadku obserwacji z nr 289 kluczowe czynniki, ktróre wpłynęły na predykcję modelu to liczba dni od rezerwacji do przyjazdu oraz typ klienta wraz z krajem z którego pochodzi. Zauważmy jednak, że typ klienta i kraj miał sprzeczny z predykcją wpływ, co jednak nie wpłynęło na końcowy rezultat. Co ciekawe, w przypadku tej obserwacji kraj pochodzenia gościa miał znikomy wpływ.

W przypadku obserwacji 999 przy przedstawionej kolejności zmiennych w tym przypadku zaskakująco duży, dominujący wpływ miało to, że gość zarezerwował miejsce parkingowe. Kolejną zmienną o sporym wpływie, jednak o rząd wielkości mniejszym, była liczba specjalnych próśb.


### 4. Dwie obserwacje, które mają dla tych samych zmiennych inne efety

In [38]:
pps_list[5].plot() ## 2626
pps_list[0].plot() ## 289

Możemy zauważyć, że w powyższych predykcjach wpływ rezerwacji na jedną noc w tygodniu jest inny. W przypadku obserwacji z nr 2626 jest to zmienna wpływająca na anulowanie rezerwacji. Jednak przy obserwacji 289 wpływ ten jest przeciwny. 